In [1]:
import os
import h5py
import numpy as np
from typing import List, Dict
from stim_pyper.processing_utils.optimizer_postprocessing import process_vta


Helper Functions

Assumes a leaddbs BIDS directory, with patients stored in the same lead group. May need to modify the get_stimulations function slightly if your dataset is organized differently.

In [ ]:
def get_stimulations(base_path, lead_group) -> List[Dict]:
    """
    Gets reconstruction files of all leaddbs patients for batch processing.
    """
    for root, dirs, _ in os.walk(path.join(base_path, 'derivatives', 'leaddbs')):
        for dir_name in dirs:
            stim = os.path.join(root, dir_name, 'stimulations', 'MNI152NLin2009bAsym', lead_group, f'{dir_name}_desc-stimparameters.mat')
            if os.path.isfile(stim):
                data = loadmat(stim, simplify_cells=True)
                json_data.append({'dir_name': dir_name, 'data': data})
    return json_data

In [ ]:
def get_reconstructions(base_path) -> List[Dict]:
    """
    Gets reconstruction files of all leaddbs patients for batch processing.
    """
    json_data = []
    for root, dirs, _ in os.walk(path.join(base_path, 'derivatives', 'leaddbs')):
        for dir_name in dirs:
            mat_file_path = os.path.join(root, dir_name, 'reconstruction', f'{dir_name}_desc-reconstruction.mat')
            if os.path.isfile(mat_file_path):
                data = loadmat(mat_file_path, simplify_cells=True)
                json_data.append({'dir_name': dir_name, 'data': data['reco']['mni']['coords_mm'][:]})
    return json_data

In [ ]:
def get_v(stim):
    try:
        with h5py.File(stim, 'r') as f:
            S = f['S']
            right_elec = S['Rs1']
            right_amp = np.array(right_elec['amp'])
            right_v = []
            for key, value in sorted(right_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
                if key.startswith('k'):
                    perc = np.array(value.get('perc', np.nan))
                    try:
                        right_v.append(right_amp * perc / 100)
                    except:
                        right_v.append(0)

            left_elec = S['Ls1']
            left_amp = np.array(left_elec['amp'])
            left_v = []
            for key, value in sorted(left_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
                if key.startswith('k'):
                    perc = np.array(value.get('perc', np.nan))
                    try:
                        left_v.append(left_amp * perc / 100)
                    except:
                        left_v.append(0)
            
            v = {'right': right_v, 'left': left_v}
    except:
            f = loadmat(stim, simplify_cells=True)
            S = f['S']
            right_elec = S['Rs1']
            right_amp = np.array(right_elec['amp'])
            right_v = []
            for key, value in sorted(right_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
                if key.startswith('k'):
                    perc = np.array(value.get('perc', np.nan))
                    try:
                        right_v.append(right_amp * perc / 100)
                    except:
                        right_v.append(0)

            left_elec = S['Ls1']
            left_amp = np.array(left_elec['amp'])
            left_v = []
            for key, value in sorted(left_elec.items(), key=lambda x: (x[0][0], int(x[0][1:])) if x[0][1:].isdigit() else (x[0][0], x[0][1:])):
                if key.startswith('k'):
                    perc = np.array(value.get('perc', np.nan))
                    try:
                        left_v.append(left_amp * perc / 100)
                    except:
                        left_v.append(0)
            
            v = {'right': right_v, 'left': left_v}
    return v

In [ ]:
def leaddbs_to_stimpyper(base_path, stimulations, reconstructions, output_dir, val=None):
    """Processes the data and calls handle_nii_map for each combination of lambda and weight."""
    for reco, stim in zip(reconstructions, stimulations):
        coords_list = ['coords1', 'coords2']
        side = ['right', 'left']
        stimdata = get_v(stim['data'])
        for i, coords in enumerate(coords_list):
            try:
                optima_ampers = np.copy(stimdata[side[i]])
                electrode_coords = np.array(reco['data'][i])
                electrode_idx = i
                out_dir = os.path.join(output_dir, reco['dir_name'], f'{side[i]}')
                process_vta(optima_ampers, electrode_coords, electrode_idx, out_dir)
            except Exception as e:
                print(f"Error processing {reco['dir_name']} for {coords}: {e}")
                continue

Start Here

In [ ]:
lead_folder = ''
out_dir = ''
lead_group = '' # 'gs_xxxxxx'
stimulations = get_stimulations(lead_folder, lead_group)
reconstructions = get_reconstructions(lead_folder)
leaddbs_to_stimpyper(lead_folder, stimulations, reconstructions, out_dir)